In [15]:
#!/usr/bin/env python3
"""
SmolLM2-135M Training from Scratch (Fully Configurable via JSON)
Features:
- Exact SmolLM2 architecture
- Loads hyperparameters from JSON config file
- SmolLM2 tokenizer from HuggingFace
- Mixed precision (BF16 for CUDA, FP16 for MPS)
- Flash Attention & RoPE
- AdamW optimizer with cosine schedule
- Checkpoint saving
- Training from input.txt
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from dataclasses import dataclass
import json, os, math
from tqdm import tqdm
from dataclasses import dataclass, fields
import json

# =============================
# Device Configuration
# =============================
def get_device_config():
    if torch.cuda.is_available():
        return torch.device("cuda"), True, torch.bfloat16, f"CUDA ({torch.cuda.get_device_name(0)})"
    elif torch.backends.mps.is_available():
        return torch.device("mps"), True, torch.float16, "Apple Silicon (MPS)"
    else:
        return torch.device("cpu"), False, torch.float32, "CPU"

# =============================
# Load config from JSON
# =============================
@dataclass
class SmolLM2Config:
    vocab_size: int = 49152
    hidden_size: int = 576
    intermediate_size: int = 1536
    num_hidden_layers: int = 30
    num_attention_heads: int = 9
    num_key_value_heads: int = 3
    max_position_embeddings: int = 8192
    rms_norm_eps: float = 1e-5
    rope_theta: float = 100000.0
    attention_dropout: float = 0.0
    hidden_dropout: float = 0.0   # ✅ default added
    initializer_range: float = 0.041666666666666664
    tie_word_embeddings: bool = True

    def __post_init__(self):
        self.head_dim = self.hidden_size // self.num_attention_heads

    @classmethod
    def from_json(cls, path):
        with open(path, "r") as f:
            cfg = json.load(f)
        allowed_keys = {f.name for f in fields(cls)}
        filtered_cfg = {k: v for k, v in cfg.items() if k in allowed_keys}
        return cls(**filtered_cfg)

# =============================
# RMSNorm
# =============================
class RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.eps = eps

    def forward(self, x):
        var = x.pow(2).mean(-1, keepdim=True)
        x = x * torch.rsqrt(var + self.eps)
        return self.weight * x

# =============================
# Rotary Position Embedding
# =============================
class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=8192, base=100000.0):
        super().__init__()
        self.dim = dim
        self.base = base
        inv_freq = 1.0 / (self.base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

    def forward(self, seq_len, device):
        t = torch.arange(seq_len, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.outer(t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        return emb.cos()[None, :, :], emb.sin()[None, :, :]

def rotate_half(x):
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2:]
    return torch.cat([-x2, x1], dim=-1)

def apply_rotary(q, k, cos, sin):
    return (q * cos + rotate_half(q) * sin, k * cos + rotate_half(k) * sin)

# =============================
# Grouped Query Attention
# =============================
class GroupedQueryAttention(nn.Module):
    def __init__(self, cfg: SmolLM2Config):
        super().__init__()
        self.cfg = cfg
        self.q_proj = nn.Linear(cfg.hidden_size, cfg.num_attention_heads * cfg.head_dim, bias=False)
        self.k_proj = nn.Linear(cfg.hidden_size, cfg.num_key_value_heads * cfg.head_dim, bias=False)
        self.v_proj = nn.Linear(cfg.hidden_size, cfg.num_key_value_heads * cfg.head_dim, bias=False)
        self.o_proj = nn.Linear(cfg.num_attention_heads * cfg.head_dim, cfg.hidden_size, bias=False)
        self.rotary = RotaryEmbedding(cfg.head_dim, max_position_embeddings=cfg.max_position_embeddings, base=cfg.rope_theta)

        self.num_key_value_groups = cfg.num_attention_heads // cfg.num_key_value_heads

    def forward(self, x):
        B, T, _ = x.shape
        q = self.q_proj(x).view(B, T, self.cfg.num_attention_heads, self.cfg.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(B, T, self.cfg.num_key_value_heads, self.cfg.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, T, self.cfg.num_key_value_heads, self.cfg.head_dim).transpose(1, 2)

        cos, sin = self.rotary(T, x.device)
        q, k = apply_rotary(q, k, cos, sin)

        # Repeat KV heads
        k = k.repeat_interleave(self.num_key_value_groups, dim=1)
        v = v.repeat_interleave(self.num_key_value_groups, dim=1)

        attn = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.cfg.attention_dropout if self.training else 0.0, is_causal=True)
        attn = attn.transpose(1, 2).contiguous().view(B, T, self.cfg.hidden_size)
        return self.o_proj(attn)

# =============================
# MLP (SwiGLU)
# =============================
class MLP(nn.Module):
    def __init__(self, cfg: SmolLM2Config):
        super().__init__()
        self.gate = nn.Linear(cfg.hidden_size, cfg.intermediate_size, bias=False)
        self.up = nn.Linear(cfg.hidden_size, cfg.intermediate_size, bias=False)
        self.down = nn.Linear(cfg.intermediate_size, cfg.hidden_size, bias=False)

    def forward(self, x):
        return self.down(F.silu(self.gate(x)) * self.up(x))

# =============================
# Transformer Decoder Layer
# =============================
class DecoderLayer(nn.Module):
    def __init__(self, cfg: SmolLM2Config):
        super().__init__()
        self.ln1 = RMSNorm(cfg.hidden_size, eps=cfg.rms_norm_eps)
        self.attn = GroupedQueryAttention(cfg)
        self.ln2 = RMSNorm(cfg.hidden_size, eps=cfg.rms_norm_eps)
        self.mlp = MLP(cfg)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

# =============================
# SmolLM2 Model
# =============================
class SmolLM2Model(nn.Module):
    def __init__(self, cfg: SmolLM2Config):
        super().__init__()
        self.cfg = cfg
        self.embed_tokens = nn.Embedding(cfg.vocab_size, cfg.hidden_size)
        self.layers = nn.ModuleList([DecoderLayer(cfg) for _ in range(cfg.num_hidden_layers)])
        self.ln_f = RMSNorm(cfg.hidden_size, eps=cfg.rms_norm_eps)

    def forward(self, input_ids):
        x = self.embed_tokens(input_ids)
        for layer in self.layers:
            x = layer(x)
        x = self.ln_f(x)
        return x

# =============================
# SmolLM2 for Causal LM
# =============================

class SmolLM2ForCausalLM(nn.Module):
    def __init__(self, config: SmolLM2Config):
        super().__init__()
        self.config = config  # ✅ Always attach the config
        self.model = SmolLM2Model(config)

        # LM head (tied or untied)
        if config.tie_word_embeddings:
            self.lm_head = None  # Will use embed_tokens.weight
        else:
            self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        std = self.config.initializer_range
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=std)

    def forward(self, input_ids, labels=None):
        hidden_states = self.model(input_ids)

        # Compute logits
        if self.config.tie_word_embeddings:
            logits = F.linear(hidden_states, self.model.embed_tokens.weight)
        else:
            logits = self.lm_head(hidden_states)

        # Compute loss if labels provided
        loss = None
        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, self.config.vocab_size),
                shift_labels.view(-1),
                ignore_index=-100
            )

        return logits, loss

    def num_parameters(self):
        return sum(p.numel() for p in self.parameters())

# =============================
# Dataset for input.txt
# =============================
class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=1024, batch_size=4):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.batch_size = batch_size
        text = open(file_path, "r", encoding="utf-8").read()
        self.data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
        tokens_per_batch = block_size * batch_size
        num_batches = len(self.data) // tokens_per_batch
        self.data = self.data[:num_batches * tokens_per_batch]
        self.num_batches = num_batches

    def __len__(self):
        return self.num_batches

    def __getitem__(self, idx):
        tokens_per_batch = self.block_size * self.batch_size
        start = idx * tokens_per_batch
        chunk = self.data[start:start + tokens_per_batch + 1]
        if len(chunk) < tokens_per_batch + 1:
            chunk = torch.cat([chunk, torch.zeros(tokens_per_batch + 1 - len(chunk), dtype=torch.long)])
        x = chunk[:-1].view(self.batch_size, self.block_size)
        y = chunk[1:].view(self.batch_size, self.block_size)
        return x, y

# =============================
# Text Generation
# =============================
def generate_sample_text(model, prompt, tokenizer, device, max_tokens=100, temperature=0.8):
    model.eval()
    context = torch.tensor(tokenizer.encode(prompt), dtype=torch.long).unsqueeze(0).to(device)
    with torch.no_grad():
        for _ in range(max_tokens):
            context_window = context[:, -model.config.max_position_embeddings:]
            logits, _ = model(context_window)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            context = torch.cat([context, next_token], dim=1)
    model.train()
    return tokenizer.decode(context[0].cpu().tolist())


# =============================
# Training Loop
# =============================
@dataclass
class TrainingConfig:
    input_file: str = "input.txt"
    block_size: int = 32
    max_steps: int = 5000
    batch_size: int = 4
    gradient_accumulation_steps: int = 16
    learning_rate: float = 6e-4
    min_lr: float = 6e-5
    warmup_steps: int = 100
    weight_decay: float = 0.1
    beta1: float = 0.9
    beta2: float = 0.95
    grad_clip: float = 1.0
    checkpoint_dir: str = "checkpoints"
    save_every: int = 500
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    use_amp: bool = True

def train():
    device, use_amp, amp_dtype, device_name = get_device_config()
    train_config = TrainingConfig()
    train_config.device = device
    train_config.use_amp = use_amp

    print(f"Device: {device_name}, Mixed precision: {use_amp}, dtype: {amp_dtype}")

    os.makedirs(train_config.checkpoint_dir, exist_ok=True)

    # Load config and tokenizer
    cfg = SmolLM2Config.from_json("config.json")
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
    dataset = TextDataset(train_config.input_file, tokenizer, train_config.block_size, train_config.batch_size)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

    model = SmolLM2ForCausalLM(cfg).to(device)

    try:
        optimizer = torch.optim.AdamW(model.parameters(), lr=train_config.learning_rate,
                                      betas=(train_config.beta1, train_config.beta2),
                                      weight_decay=train_config.weight_decay, fused=True)
    except:
        optimizer = torch.optim.AdamW(model.parameters(), lr=train_config.learning_rate,
                                      betas=(train_config.beta1, train_config.beta2),
                                      weight_decay=train_config.weight_decay)

    # Preload all batches
    all_batches = [b for b in dataloader]

    global_step = 0
    micro_batch_count = 0
    optimizer.zero_grad()
    pbar = tqdm(total=train_config.max_steps, desc="Training", dynamic_ncols=True)

    while global_step < train_config.max_steps:
        x, y = all_batches[micro_batch_count % len(all_batches)]
        x, y = x.squeeze(0).to(device), y.squeeze(0).to(device)

        if use_amp:
            with torch.autocast(device.type, dtype=amp_dtype):
                logits, loss = model(x, labels=y)
                loss = loss / train_config.gradient_accumulation_steps
        else:
            logits, loss = model(x, labels=y)
            loss = loss / train_config.gradient_accumulation_steps

        loss.backward()
        micro_batch_count += 1

        if micro_batch_count % train_config.gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), train_config.grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

            # Progress bar
            pbar.update(1)
            pbar.set_postfix({'loss': f'{loss.item() * train_config.gradient_accumulation_steps:.4f}'})

            # Generate text every 50 steps
            if global_step % 50 == 0:
                print(f"\n--- Sample Generation at Step {global_step} ---")
                prompts = ["The ", "In ", "To ", "A "]
                for prompt in prompts:
                    print(f"Prompt: '{prompt}'")
                    try:
                        print(generate_sample_text(model, prompt, tokenizer, device))
                    except Exception as e:
                        print(f"Generation failed: {e}")
                print("-" * 80)

            # Save checkpoint every 500 steps
            if global_step % train_config.save_every == 0:
                checkpoint_path = os.path.join(train_config.checkpoint_dir, f"checkpoint_step_{global_step}.pt")
                torch.save({
                    'global_step': global_step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'config': cfg
                }, checkpoint_path)
                print(f"\n✓ Checkpoint saved: {checkpoint_path}")

    pbar.close()
    print("\nTraining complete!")

if __name__ == "__main__":
    train()

Device: CUDA (Tesla T4), Mixed precision: True, dtype: torch.bfloat16


Token indices sequence length is longer than the specified maximum sequence length for this model (341094 > 8192). Running this sequence through the model will result in indexing errors

Training:   1%|          | 50/5000 [01:59<3:14:14,  2.35s/it, loss=6.5151]


--- Sample Generation at Step 50 ---
Prompt: 'The '
The 
 must: death thy my,st despair
 in andB inAnd and of, learn
 lord OF: your's, to and's: and,,Or
ENARDESS
, thereofTh,ge love, storyLong to,,
's,I: is
 again jealous her in and and, in:ates my the she
 dear danger, to rough
est in a:

blown
 no: the to here
 children my soul desireI thy with.
Prompt: 'In '
In 
AB: IING am of,,R.
ICH
ICH
 R: shall, daily:If toSoThat no,, II thy, the,'s I: meAMAM
 to Ifor,,, qu, go hath mayful to's, to of when.
 leads gro their,: wrong thy, let hast of,, then and in,,ING's and with my me. Richard, thou;, every never ChristAB us, resign,
Prompt: 'To '
To 
 to in the;: what and and close
 thyV there;!
ING have and is
 my the the untoHow, GodThe a and is, I; a make will to in
:: humour
--ity in, coward here, is my royal now walls
 not and,, on my and
ful of would with bes the I.
 R in isfort, by not
 R this I bless, I loveill
EN'sR-: my his
Prompt: 'A '
A  is
, to live tid fame,, ap conscience theirse


Training:   2%|▏         | 100/5000 [04:11<3:11:40,  2.35s/it, loss=5.3908]


--- Sample Generation at Step 100 ---
Prompt: 'The '
The  how move and,And can,AndAnd breath
 that by my they in crown
 so common.
IFD a tears were and strong, the w
 from's and the and and, me give with
'sAD cannot prison black:And his to.
INGICH
,'s yielding and.
W, I ste:But's to,That for for by.
INGKKINGINGINGINGINGWARD
KNALadeAR:
 me can both will
Prompt: 'In '
In  sl,For child comfort come
 words toHave, I love an thou's light
 I have not like hungry with and be,Which?
KESTGAR: is watch and, thy father howAndAndAndAndFor
 that this father, thou's I.
KINGKW:Too a thy stand,With,Stay in stay.
KW passages that thy God.
KINGWARD:And; fiery wilt with,Andine I end thy.

Prompt: 'To '
To  to thousand his to save,AndAndAnd,And,AndAnd entrance speakFivele.
W words'smbling,, we through for tongue,Hold
 in if one I him in conference
s my and his a away?That
KWARD and thy to promise me his
 murder forless your brother that their my
, hoursan endar I's with and,Is thy a aded my!
KKou bo usla

...


--- Sample Generation at Step 4950 ---
Prompt: 'The '
The  is' which to wild an intent
rem, toit fresh in mighty, on n,With
 lay dam royalty and's: was thy,Very,
 where stand, a of title, all news
 in moreTo with. do like howoth been old
 to are gentlemen there
 got, soon foron you been violent,
 for dancing.
 half great, de! have me your.
 what Lady! whyvest!
PAINA--
's heads alter.

Prompt: 'In '
In , many prince why season brother,
 have not'd secretStill: islets son
akes! not much fear her hear but her shows
 I go her: is not blem cannot
 some noble, that have honour it
 him go her to their: your is of
akes that soul be of own thee 's much
re, one have enough some: he of, a, cureed
 humism of country else not of country are
ellowsour; her,, think is
Prompt: 'To '
To  to, she aan die this:, why labour play
 honestyerst her her the title
 that., the of, on the's,
 there to!
PAINA--The wasof these-ong look at!
POEIS kind other! how this wise procure!
ISELOW
 was prince but swift!,,!,


Training: 100%|██████████| 5000/5000 [3:44:33<00:00,  2.48s/it, loss=0.3623]


--- Sample Generation at Step 5000 ---
Prompt: 'The '
The  find find with and power love.
First:F
 told lie mother good; would my cannot ac.
A for! amen this ca indeed--
SIN EDARD
 mine--
ist it,?,iltent away
 shallow bid,,,ost knowilyatter,WhenHe coming thy heart thy by,
 told my and are, I-- you I?
ent of.
First,LEES
's resolve on my,am,lab: I lean
Prompt: 'In '
In , p, I atThey a manal.
MARUSY
Tor
How yout
 division I to piece sore you sir
 with one, to but is end way
 I thy and b you Hortio
 accuse daily
It bearsray in earnest a he done?
MARUS
SINENT:I not the it;We beenble the in part?
SINUS
 have ro thousandthough mountain: is I, back one uncle in a.

Prompt: 'To '
To  alone so yet welcome I: was own
 of power what is, mistake or accuse
 hell person so, never upon head in
 his
 of, all shall him hisers,
 cannot it the did out a day neer,That I accuse
 most all ears hate was first meught more
an till little
 a e I, death I, a to to him your father
all tyrann gone he, will I, thin

Training: 100%|██████████| 5000/5000 [3:45:23<00:00,  2.70s/it, loss=0.3623]


✓ Checkpoint saved: checkpoints/checkpoint_step_5000.pt

Training complete!
